In [1]:
# This line solves some minor problems when you do not have propery set the PYTHONPATH
exec(compile(open("./fix_paths.py", "rb").read(), "./fix_paths.py", 'exec'))

import numpy as np
import scipy.ndimage.filters as fi
import pandas as pd
import time
from concurrent import futures 
from common import dataset_loaders
import os
import settings
from keras.callbacks import History


Using TensorFlow backend.


In [7]:
batch_size = 1
big_batch_size_train, big_batch_size_valid = 20, 5

filepath= '/home/jose/tech/ml_projects/data_models/sealions_contest/new_dataset_only_lions'

experiment_folder_name = 'seal_finder_unet'
experiment_name = 'independent_classes' # This one will only be used for the logs
OUTPUT_MODEL = '%s/%s/models/independent_classes.hdf5' % (settings.DATAMODEL_PATH, experiment_folder_name)



os.system('mkdir -p %s/%s' % (settings.DATAMODEL_PATH, experiment_folder_name))
os.system('mkdir -p %s/%s/models' % (settings.DATAMODEL_PATH, experiment_folder_name))
os.system('mkdir -p %s/%s/logs' % (settings.DATAMODEL_PATH, experiment_folder_name))

# What cases are there available for training?
casenames = list(set([filename.split(".")[0].split("_")[2] for filename in os.listdir(filepath)]))

# Split in train test (hardcoded so far)
casenames = sorted(casenames)
train_casenames = casenames[:-40]
valid_casenames = casenames[-40:]
train_cases = ["%s"%(filename) for filename in os.listdir(filepath) if filename.split("_")[2].split(".")[0] in train_casenames]
valid_cases = ["%s"%(filename) for filename in os.listdir(filepath) if filename.split("_")[2].split(".")[0] in valid_casenames]

loss_history = History()


def load_case(filename):
    os.system('tar -xzvf %s/%s' % (filepath, filename))
    f = open('home/jose/tech/ml_projects/data_models/sealions_contest/new_dataset_only_lions/%s' % filename.split(".")[0], 'rb')
    img = np.load(f)
    labs = np.load(f)
    f.close()
    os.system('rm -rf home')  
    return img, labs

def get_generator(filenames, batch_size, img_shape = 512):
    
    def fill_padding(img, labs, img_shape = 512):
        # Input shape: (5, 500, 500, 3) (5, 5, 500, 500)
        img_pl = np.zeros([img.shape[0], img_shape, img_shape, img.shape[3]])
        labs_pl = np.zeros([labs.shape[0], labs.shape[1], img_shape, img_shape])
        img_pl[:,:img.shape[1],:img.shape[2], :] = img
        labs_pl[:,:,:labs.shape[2],:labs.shape[3]] = labs
        return img_pl.transpose(0,3,1,2), labs_pl
    
    X,Y = [], []
    while(1):
        sample_now = np.random.randint(len(filenames))
        lab, img = load_case(filenames[sample_now])
        if lab.shape == (500, 500) and img.shape == (500, 500, 3):
            X.append(img)
            Y.append(np.array([lab == i for i in range(5)]))
        else:
            print('Wrong shape image: ', sample_now, lab.shape, img.shape)

        if len(X) == batch_size:
            Xs, Ys = np.array(X, dtype = np.float32), np.array(Y, dtype = np.int32)
            yield fill_padding(Xs, Ys, img_shape = img_shape)
            X, Y = [], []

def plot_img_labs(img, labs):
    a,b = img, labs
    from pylab import figure, imshow, subplot
    %pylab inline
    figure(figsize(20,20))

    for i in range(5):
        subplot(321+i)
        aux = np.copy(a)
        aux[aux != i] = np.nan
        imshow(b)
        imshow(aux, alpha = 1)

        
        
train_generator = get_generator(train_cases, big_batch_size_train)
valid_generator = get_generator(valid_cases, big_batch_size_valid)

In [8]:
from dl_utils.dl_networks.unet import ThickUNET
model = ThickUNET(input_shape=(3,512,512), activation='relu', init='glorot_normal', saved_file=None,  pos_weight=100, num_labels = 5)

/home/jose/tech/ml_projects/kaggle/sealions_contest/dl_utils/dl_networks/unet.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same", kernel_initializer="glorot_normal")`
  conv1 = Convolution2D(first_depth, 3, 3, activation=activation, init=init, border_mode='same')(inputs)
/home/jose/tech/ml_projects/kaggle/sealions_contest/dl_utils/dl_networks/unet.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same", kernel_initializer="glorot_normal")`
  conv1 = Convolution2D(first_depth, 3, 3, activation=activation, init=init, border_mode='same')(conv1)
/home/jose/tech/ml_projects/kaggle/sealions_contest/dl_utils/dl_networks/unet.py:47: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same", kernel_initializer="glorot_normal")`
  conv2 = Convolution2D(first_depth*2, 3, 3, activation=activation, init=init, bord

In [10]:

nb_epoch=1000
verbose=1
#class_weight={0:1., 1:4.},
validation_steps_per_epoch = 1
train_steps_per_epoch      = 1


min_val_loss = 10000
#try:
if True:
    for i_epoch in range(nb_epoch):
        j_ep = 0

        t1 = time.time()
        loss_train = []
        for x, y in train_generator:
            if j_ep  < train_steps_per_epoch:
                j_ep += 1
                model.model.fit(x,y,verbose = 0, batch_size=batch_size,epochs=1,shuffle = False, callbacks=[loss_history])
                loss_train.append(np.mean(loss_history.history['loss']))
            else:
                break
        train_loss = np.mean(loss_train)

        # Validation stage
        losses = []
        j_valid = 0
        for x,y in valid_generator:
            if j_valid < validation_steps_per_epoch:
                losses.append( model.model.evaluate(x,y, verbose=0))
                j_valid += 1
            else:
                break
        valid_loss = np.mean(losses)

        # Save weights
        if valid_loss < min_val_loss:
            min_val_loss = valid_loss
            model.model.save_weights(OUTPUT_MODEL)
        print("Epoch %d   -  valid loss: %0.3f   -   train loss: %0.3f    - Time %0.2f" % (i_epoch, valid_loss, train_loss, time.time()-t1))
#except:
#    model.save_weights(OUTPUT_MODEL.split(".")[0] + "_interrupted_by_exception.hdf5")

print("EXITING!")


Wrong shape image:  5748 (244, 500) (244, 500, 3)
Epoch 0   -  valid loss: 0.911   -   train loss: 0.888    - Time 33.35
Wrong shape image:  4314 (328, 500) (328, 500, 3)
Wrong shape image:  4756 (244, 500) (244, 500, 3)
Wrong shape image:  811 (328, 500) (328, 500, 3)
Epoch 1   -  valid loss: 0.999   -   train loss: 0.904    - Time 32.64
Wrong shape image:  4756 (244, 500) (244, 500, 3)
Wrong shape image:  152 (500, 492) (500, 492, 3)
Epoch 2   -  valid loss: 0.771   -   train loss: 0.855    - Time 32.72
Wrong shape image:  371 (500, 116) (500, 116, 3)
Epoch 3   -  valid loss: 0.783   -   train loss: 0.848    - Time 33.29
Wrong shape image:  5879 (328, 492) (328, 492, 3)
Wrong shape image:  692 (328, 500) (328, 500, 3)
Wrong shape image:  2821 (328, 500) (328, 500, 3)
Epoch 4   -  valid loss: 0.778   -   train loss: 0.841    - Time 33.49
Wrong shape image:  1636 (244, 500) (244, 500, 3)
Epoch 5   -  valid loss: 0.810   -   train loss: 0.855    - Time 33.33
Wrong shape image:  210 (500

KeyboardInterrupt: 